# HTTP Requests Analysis
- This notebook demonstrates basic use of `http_requests` table and `req_call_stack` columns.
- `req_call_stack` column contains the JavaScript call stack for requests that are initiated by scripts.
- Based on https://github.com/mozilla/openwpm-crawler/blob/master/analysis/Sample%20Analysis.ipynb


In [1]:
import re
import json
import sqlite3
import pandas as pd

In [2]:
# import some analysis utilities from https://github.com/englehardt/crawl_utils
import sys
sys.path.append('./crawl_utils/')
import domain_utils as du
import analysis_utils as au

In [3]:
# use the sample sqlite for the 2018-06 stateless crawl
DB = 'sample_2018-06_1m_stateless_census_crawl.sqlite'

### Load the requests

In [4]:
# Load the data
con = sqlite3.connect(DB)
con.row_factory = sqlite3.Row
cur = con.cursor()
reqs = pd.read_sql_query("SELECT * FROM http_requests", con)

In [5]:
total_sites = reqs['top_level_url'].nunique()
print "Database contains %s HTTP requests on %d sites" %(len(reqs), total_sites)

Database contains 156285 HTTP requests on 944 sites


### Add additional columns to help with analysis

In [6]:
# Add the public suffix + 1 of a bunch of the URL columns
reqs['url_ps1'] = reqs['url'].apply(du.get_ps_plus_1)
reqs['top_ps1'] = reqs['top_level_url'].apply(lambda x: du.get_ps_plus_1(x) if x is not None else None)
reqs['loading_ps1'] = reqs['loading_href'].apply(lambda x: du.get_ps_plus_1(x) if x is not None else None)

In [7]:
# Parse some info out of the call stack
# req_call_stack column contains the JavaScript stack frames for requests initiated by scripts
reqs['stack_scripts'] = reqs['req_call_stack'].apply(au.get_script_urls_from_call_stack_as_set)
reqs['stack_ps1s'] = reqs['stack_scripts'].apply(lambda x: set([du.get_ps_plus_1(y) for y in x]))

### How many sites is doubleclick.net loaded on?

In [8]:
n_sites = reqs[reqs['url_ps1'] == 'doubleclick.net'].top_level_url.nunique()
print "%d sites (%0.1f%%)" % (n_sites, 100 * n_sites / float(total_sites))

581 sites (61.5%)


### How many sites is google-analytics.com loaded on?

In [9]:
n_sites = reqs[reqs['url_ps1'] == 'google-analytics.com'].top_level_url.nunique()
print "%d sites (%0.1f%%)" % (n_sites, 100 * n_sites / float(total_sites))

535 sites (56.7%)


### How many sites is facebook.com loaded on?

In [10]:
n_sites = reqs[reqs['url_ps1'] == 'facebook.com'].top_level_url.nunique()
print "%d sites (%0.1f%%)" % (n_sites, 100 * n_sites / float(total_sites))

398 sites (42.2%)


### What domains does doubleclick.net load other resources from?

In [11]:
reqs[
    reqs.stack_ps1s.apply(lambda x: 'doubleclick.net' in x)
].groupby('url_ps1').top_level_url.count().sort_values(ascending=False)

url_ps1
doubleclick.net                  2184
googlesyndication.com            1988
google.com                        335
moatads.com                       159
rubiconproject.com                128
2mdn.net                          100
adnxs.com                          87
googletagservices.com              82
adsafeprotected.com                55
advertising.com                    48
casalemedia.com                    35
openx.net                          35
amazon-adsystem.com                35
gstatic.com                        33
clarium.global.ssl.fastly.net      31
agkn.com                           28
trugaze.io                         25
demdex.net                         23
biobiochile.cl                     21
criteo.com                         21
detik.net.id                       20
asahicom.jp                        20
doubleverify.com                   20
quantserve.com                     19
pubmatic.com                       19
almasryalyoum.com                  19
sasc